In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
import os
import gc
import random

from tqdm.auto import tqdm
import torch.nn.functional as F
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from segmentation_models_pytorch.losses import DiceLoss

In [2]:
def seed_everything(seed=100):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything()

In [57]:
# Select the device to train on
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
EPOCHS = 400
LR = 1e-4
p = 0.3
IMG_SIZE = 192
BATCH_SIZE = 16

In [4]:
images = os.listdir("NR206/train")
masks = os.listdir("NR206/train_labels")
train_df = pd.DataFrame({"images": map(lambda x: os.path.join("NR206/train", x), images),
                         "masks": map(lambda x: os.path.join("NR206/train_labels", x), masks)})
print(train_df.shape)
del images, masks
train_df.head()

(126, 2)


,images,masks
0,NR206/train\NORMAL1.png,NR206/train_labels\NORMAL1.png
1,NR206/train\NORMAL10.png,NR206/train_labels\NORMAL10.png
2,NR206/train\NORMAL100.png,NR206/train_labels\NORMAL100.png
3,NR206/train\NORMAL101.png,NR206/train_labels\NORMAL101.png
4,NR206/train\NORMAL102.png,NR206/train_labels\NORMAL102.png


In [5]:
images = os.listdir("NR206/val")
masks = os.listdir("NR206/val_labels")
val_df = pd.DataFrame({"images": map(lambda x: os.path.join("NR206/val", x), images),
                       "masks": map(lambda x: os.path.join("NR206/val_labels", x), masks)})
print(val_df.shape)
del images, masks
val_df.head()

(40, 2)


,images,masks
0,NR206/val\NORMAL129.png,NR206/val_labels\NORMAL129.png
1,NR206/val\NORMAL131.png,NR206/val_labels\NORMAL131.png
2,NR206/val\NORMAL133.png,NR206/val_labels\NORMAL133.png
3,NR206/val\NORMAL135.png,NR206/val_labels\NORMAL135.png
4,NR206/val\NORMAL137.png,NR206/val_labels\NORMAL137.png


In [6]:
images = os.listdir("NR206/test")
masks = os.listdir("NR206/test_labels")
test_df = pd.DataFrame({"images": map(lambda x: os.path.join("NR206/test", x), images),
                        "masks": map(lambda x: os.path.join("NR206/test_labels", x), masks)})
print(test_df.shape)
del images, masks
test_df.head()

(40, 2)


,images,masks
0,NR206/test\NORMAL112.png,NR206/test_labels\NORMAL112.png
1,NR206/test\NORMAL121.png,NR206/test_labels\NORMAL121.png
2,NR206/test\NORMAL125.png,NR206/test_labels\NORMAL125.png
3,NR206/test\NORMAL14.png,NR206/test_labels\NORMAL14.png
4,NR206/test\NORMAL15.png,NR206/test_labels\NORMAL15.png


In [7]:
def encode_labels(masks):
    labelencoder = LabelEncoder()
    masks_reshaped = masks.reshape(-1)
    masks_encoded = labelencoder.fit_transform(masks_reshaped)
    del masks_reshaped, labelencoder
    return masks_encoded.reshape(masks.shape)

In [8]:
# Create a custom dataset class
classes = 9


class SegmentationDataset(Dataset):
    def __init__(self, df):
        self.df = df
        images = []
        masks = []
        for idx in range(len(self.df)):
            sample = self.df.iloc[idx]
            images.append(np.array(load_img(sample.images, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE))))
            masks.append(np.array(load_img(sample.masks, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE))))
        images = np.array(images)
        masks = np.array(masks)
        n_classes = classes  #len(np.unique(masks))
        masks_encoded = encode_labels(masks)
        self.masks_cat = to_categorical(masks_encoded, num_classes=n_classes)
        self.images = np.expand_dims(images, axis=-1) / 255.0
        del masks_encoded, images, masks, n_classes

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = torch.from_numpy(np.transpose(self.images[idx], (2, 0, 1))).float()
        mask = torch.from_numpy(np.transpose(self.masks_cat[idx], (2, 0, 1))).float()
        return image, mask

In [9]:
# Processed train and validation sets
train_data = SegmentationDataset(train_df)
val_data = SegmentationDataset(val_df)
test_data = SegmentationDataset(test_df)

In [10]:
print(f"Size of Trainset : {len(train_data)}")
print(f"Size of Validset : {len(val_data)}")

Size of Trainset : 126
Size of Validset : 40


In [11]:
from torch.utils.data import DataLoader

In [12]:
trainloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
testloader = DataLoader(test_data, batch_size=1, shuffle=False)

print(f"Total number of batches in Train Loader: {len(trainloader)}")
print(f"Total number of batches in Val Loader: {len(valloader)}")

Total number of batches in Train Loader: 8
Total number of batches in Val Loader: 3


In [13]:
for image, mask in trainloader:
    print(f"Size of one batch of images: {image.shape}")
    print(f"Size of one batch of masks: {mask.shape}")
    break

Size of one batch of images: torch.Size([16, 1, 192, 192])
Size of one batch of masks: torch.Size([16, 9, 192, 192])


In [14]:
dice_loss = DiceLoss(mode="multilabel", classes=9)


def dice_score(y_true, y_pred):
    return 1.0 - dice_loss(y_true, y_pred)

In [58]:
class UNet(nn.Module):
    def __init__(self, n_classes=9, img_channels=1):
        super(UNet, self).__init__()

        self.conv1 = self.double_conv(img_channels, 16)
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = self.double_conv(16, 32)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = self.double_conv(32, 64)
        self.pool3 = nn.MaxPool2d(2)

        self.conv4 = self.double_conv(64, 128)
        self.pool4 = nn.MaxPool2d(2)

        self.conv5 = self.double_conv(128, 256)

        self.up6 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv6 = self.double_conv(256, 128)

        self.up7 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv7 = self.double_conv(128, 64)

        self.up8 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.conv8 = self.double_conv(64, 32)

        self.up9 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.conv9 = self.double_conv(32, 16)

        self.out_conv = nn.Conv2d(16, n_classes, kernel_size=1)

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Dropout(p),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool1(c1)

        c2 = self.conv2(p1)
        p2 = self.pool2(c2)

        c3 = self.conv3(p2)
        p3 = self.pool3(c3)

        c4 = self.conv4(p3)
        p4 = self.pool4(c4)

        c5 = self.conv5(p4)

        u6 = self.up6(c5)
        u6 = torch.cat([u6, c4], dim=1)
        c6 = self.conv6(u6)

        u7 = self.up7(c6)
        u7 = torch.cat([u7, c3], dim=1)
        c7 = self.conv7(u7)

        u8 = self.up8(c7)
        u8 = torch.cat([u8, c2], dim=1)
        c8 = self.conv8(u8)

        u9 = self.up9(c8)
        u9 = torch.cat([u9, c1], dim=1)
        c9 = self.conv9(u9)

        outputs = self.out_conv(c9)
        return outputs

In [59]:
model = UNet()
model.to(DEVICE);

In [60]:
def train_model(data_loader, model, optimizer):
    total_loss = 0.0
    total_score = 0.0
    model.train()

    for images, masks in tqdm(data_loader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        optimizer.zero_grad()
        logits = model(images)

        loss = dice_loss(logits, masks)
        score = dice_score(logits, masks)

        loss.backward()
        optimizer.step()
        total_score += score.item()
        total_loss += loss.item()

    return total_loss / len(data_loader), total_score / len(data_loader)

In [61]:
def eval_model(data_loader, model):
    total_loss = 0.0
    total_score = 0.0
    model.eval()

    with torch.no_grad():
        for images, masks in tqdm(data_loader):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            logits = model(images)

            loss = dice_loss(logits, masks)
            score = dice_score(logits, masks)

            total_score += score.item()
            total_loss += loss.item()

        return total_loss / len(data_loader), total_score / len(data_loader)

In [62]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)

In [63]:
gc.collect()

761

In [64]:
best_val_loss = 1e9
writer = SummaryWriter(f"runs/noisy_{p}")
for i in range(EPOCHS):
    train_loss, train_score = train_model(trainloader, model, optimizer)
    val_loss, val_score = eval_model(valloader, model)

    if val_loss < best_val_loss:
        print("MODEL SAVED")

        best_val_loss = val_loss
    writer.add_scalar("Loss/train", train_loss, i)
    writer.add_scalar("Loss/val", val_loss, i)
    writer.add_scalar("Score/train", train_score, i)
    writer.add_scalar("Score/val", val_score, i)
    writer.flush()
    print(
        f"\033[1m\033[92m Epoch {i} Train DiceLoss {train_loss} Val DiceLoss {val_loss} Train dice_score {train_score} Val dice_score {val_score}")
writer.close()

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

MODEL SAVED
 Epoch 0 Train DiceLoss 0.898029699921608 Val DiceLoss 0.8963162104288737 Train dice_score 0.10197030007839203 Val dice_score 0.1036837895711263


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

 Epoch 398 Train DiceLoss 0.1578036341816187 Val DiceLoss 0.16302086412906647 Train dice_score 0.8421963751316071 Val dice_score 0.8369791309038798


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 Epoch 399 Train DiceLoss 0.160843838006258 Val DiceLoss 0.16208731631437936 Train dice_score 0.8391561508178711 Val dice_score 0.8379126787185669


In [65]:
torch.save(model.state_dict(), f'Noise_DiceLoss_SD_new_{p}.pt')
torch.save(model, f'Noise_DiceLoss_new_{p}.pt')